In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Summarize messages

In [2]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import SummarizationMiddleware
from langchain_ollama import ChatOllama

model = ChatOllama(model="lfm2.5-thinking")
model2 = ChatOllama(model="lfm2.5-thinking")

agent = create_agent(
    model=model,
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model=model2,
            trigger=("tokens", 100),
            keep=("messages", 1)
        )
    ],
)

In [3]:
from langchain.messages import HumanMessage, AIMessage
from pprint import pprint

response = agent.invoke(
    {"messages": [
        HumanMessage(content="What is the capital of the moon?"),
        AIMessage(content="The capital of the moon is Lunapolis."),
        HumanMessage(content="What is the weather in Lunapolis?"),
        AIMessage(content="Skies are clear, with a high of 120C and a low of -100C."),
        HumanMessage(content="How many cheese miners live in Lunapolis?"),
        AIMessage(content="There are 100,000 cheese miners living in Lunapolis."),
        HumanMessage(content="Do you think the cheese miners' union will strike?"),
        AIMessage(content="Yes, because they are unhappy with the new president."),
        HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?"),
        ]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'messages': [HumanMessage(content='Here is a summary of the conversation to date:\n\n## SESSION INTENT  \nThe user seeks information about the capital of the moon and related details.  \n\n## SUMMARY  \nThe capital of the moon is Lunapolis; 100,000 cheese miners reside there, and the union will strike due to dissatisfaction with the new president.  \n\n## ARTIFACTS  \nNone.  \n\n## NEXT STEPS  \nNone.', additional_kwargs={'lc_source': 'summarization'}, response_metadata={}, id='fbad275b-8bf5-412f-995f-70a32e8eb11a'),
              HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?", additional_kwargs={}, response_metadata={}, id='4b48f8c7-c1c0-4953-8579-64be688438f0'),
              AIMessage(content='As Lunapolis\' new president, I would respond to the cheese miners\' union with a blend of empathy, clarity, and resolve. Here’s how I’d approach it:  \n\n1. **Acknowledge Concerns**: "I understand the frustration these workers f

In [4]:
print(response["messages"][0].content)

Here is a summary of the conversation to date:

## SESSION INTENT  
The user seeks information about the capital of the moon and related details.  

## SUMMARY  
The capital of the moon is Lunapolis; 100,000 cheese miners reside there, and the union will strike due to dissatisfaction with the new president.  

## ARTIFACTS  
None.  

## NEXT STEPS  
None.


## Trim/delete messages

In [5]:
from typing import Any
from langchain.agents import AgentState
from langchain.messages import RemoveMessage
from langgraph.runtime import Runtime
from langchain.agents.middleware import before_agent
from langchain.messages import ToolMessage

@before_agent
def trim_messages(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Remove all the tool messages from the state"""
    messages = state["messages"]

    tool_messages = [m for m in messages if isinstance(m, ToolMessage)]
    
    return {"messages": [RemoveMessage(id=m.id) for m in tool_messages]}

In [6]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="lfm2.5-thinking")

agent = create_agent(
    model=model,
    checkpointer=InMemorySaver(),
    middleware=[trim_messages],
)

In [7]:
response = agent.invoke(
    {"messages": [
        HumanMessage(content="My device won't turn on. What should I do?"),
        ToolMessage(content="blorp-x7 initiating diagnostic ping…", tool_call_id="1"),
        AIMessage(content="Is the device plugged in and turned on?"),
        HumanMessage(content="Yes, it's plugged in and turned on."),
        ToolMessage(content="temp=42C voltage=2.9v … greeble complete.", tool_call_id="2"),
        AIMessage(content="Is the device showing any lights or indicators?"),
        HumanMessage(content="What's the temperature of the device?")
        ]},
    {"configurable": {"thread_id": "2"}}
)

pprint(response)

{'messages': [HumanMessage(content="My device won't turn on. What should I do?", additional_kwargs={}, response_metadata={}, id='1a3c0f38-4a01-41b1-9339-1d9077dc3422'),
              AIMessage(content='Is the device plugged in and turned on?', additional_kwargs={}, response_metadata={}, id='c161ddbb-a000-4d46-8081-c7c2ec58942f', tool_calls=[], invalid_tool_calls=[]),
              HumanMessage(content="Yes, it's plugged in and turned on.", additional_kwargs={}, response_metadata={}, id='a6dbcbdd-39b4-4ff8-8095-31c01b0b75f8'),
              AIMessage(content='Is the device showing any lights or indicators?', additional_kwargs={}, response_metadata={}, id='44a2a906-fa1a-4248-8cc1-69ceb22b3fb9', tool_calls=[], invalid_tool_calls=[]),
              HumanMessage(content="What's the temperature of the device?", additional_kwargs={}, response_metadata={}, id='e99272d3-1db5-4771-a246-767e57df0b41'),
              AIMessage(content='If your device isn’t turning on despite being plugged in and p

In [8]:
print(response["messages"][-1].content)

If your device isn’t turning on despite being plugged in and powered on, here are steps to troubleshoot:

### 1. **Check Power Source**  
   - Ensure the outlet is working (test with another device).  
   - Confirm the power adapter is intact and properly connected.  
   - Try a different power source (e.g., a different outlet, power strip, or charger).  

### 2. **Test the Device**  
   - Plug the device back in briefly to see if it powers on.  
   - Try restarting the device (unplug it for 10 seconds, then plug it back in).  

### 3. **Inspect Connections**  
   - Check for loose cables, ports, or connections.  
   - Ensure the device isn’t damaged or obstructing power flow.  

### 4. **Check for Software Issues**  
   - Restart the device or perform a factory reset (if applicable).  
   - Update software/firmware if available.  

### 5. **Physical Damage**  
   - If the device is old or damaged, it might not function. Consider replacing it.  

### 6. **Contact Support**  
   - If no